<a href="https://colab.research.google.com/github/STKalinowski/EncoderDecoderModelBetterGen/blob/main/t5DecoderInputInfluence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
Looking at how to imporve generation by modifying & seperating parts of the decoder encoder input.
Models like T5 are encoder-decoder models. If we want to prompt a structure, shouldn't that part only be in the decoder part?

We can understand more about encoder-decoder architecture here: https://huggingface.co/blog/encoder-decoder#encoder-decoder
'''

"\nLooking at how to imporve generation by modifying & seperating parts of the decoder encoder input.\nModels like T5 are encoder-decoder models. If we want to prompt a structure, shouldn't that part only be in the decoder part?\n"

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.3 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig
import torch
modelName = 'google/flan-t5-base'

In [4]:
model = AutoModelForSeq2SeqLM.from_pretrained(modelName)
tokenizer = AutoTokenizer.from_pretrained(modelName)


In [5]:
input = 'Summarize the text: </s> Once upon a time'
tokInput = tokenizer.encode(input, return_tensors='pt')
tokInput
decodeStart = tokenizer.encode('<pad>', return_tensors='pt')
genConf = GenerationConfig.from_pretrained(modelName, topp=0.97, temperature=0.7, do_sample=True, max_new_tokens=500)
print(genConf)

GenerationConfig {
  "_from_model_config": true,
  "decoder_start_token_id": 0,
  "do_sample": true,
  "eos_token_id": 1,
  "max_new_tokens": 500,
  "pad_token_id": 0,
  "temperature": 0.7,
  "transformers_version": "4.30.2"
}



In [ ]:
tokenizer.special_tokens_map

#####

In [16]:
# Define the input sequence and initial decoded sequence
input_sequence = f"Outline a story based in a fantasy land. There is a wizard, a warrior, and a bard. They set out for an adventure."
decoded_sequence = "Outline:\n Beginning: "
genConf = GenerationConfig.from_pretrained(modelName, top_p=0.95, temperature=0.5, do_sample=True, max_length=750, max_new_tokens=500, repetition_penalty=1.4)
print(genConf)

# Normal Version
input_ids = tokenizer.encode(input_sequence, return_tensors='pt', add_special_tokens=False)
normalOutput = model.generate(input_ids, generation_config=genConf)
normalRes = tokenizer.decode(normalOutput[0])

# Normal with Decoder Part Included
input_ids = tokenizer.encode(input_sequence+decoded_sequence, return_tensors='pt', add_special_tokens=False)
normalDecOutput = model.generate(input_ids, generation_config=genConf)
normalDecRes = tokenizer.decode(normalDecOutput[0])

# Decoder influenced
input_ids = tokenizer.encode(input_sequence, return_tensors='pt')
decoder_input_ids = tokenizer.encode(decoded_sequence, return_tensors='pt', add_special_tokens=False)
decodeOutput = model.generate(input_ids, decoder_input_ids=decoder_input_ids, generation_config=genConf)
decodeRes = tokenizer.decode(decodeOutput[0])

print(normalRes)
print(normalDecRes)
print(decodeRes)

GenerationConfig {
  "_from_model_config": true,
  "decoder_start_token_id": 0,
  "do_sample": true,
  "eos_token_id": 1,
  "max_length": 750,
  "max_new_tokens": 500,
  "pad_token_id": 0,
  "repetition_penalty": 1.4,
  "temperature": 0.5,
  "top_p": 0.95,
  "transformers_version": "4.30.2"
}

<pad> The Lord of the Rings is a fairy tale about a young girl who meets a magical wizard and is sent to a magical ring. He finds the ring and finds out that it is made of wood, but it is destroyed by a demon. He is then brought back to his home and is reunited with his father. He is sent to a magical castle, where he is greeted by the beautiful princess of the forest. She is given the chance to meet the wizard and the warrior, who are both wizards. They become friends. The wizard is a wizard, and the warrior is a warrior. The wizard takes his place in the castle. The wizard takes the ring and teaches the young witch to take it. She is given the chance to meet the demon. The wizard is a wizard, a